In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import os
import time

from FSCNN import Net

global case_study

# Select case study
# case_study = 'ITSC'
case_study = 'CWRU'
# case_study = 'Ottawa'

def train_val_test_split(data, labels, split_ratio):
    n_samples = data.shape[0]
    div1 = int(n_samples * split_ratio)
    div2 = div1 + (n_samples - div1) // 2
    indices = list(range(n_samples))

    if case_study == 'CWRU':
        test_ind_file = 'data_test_ind_CWRU.csv'
    elif case_study == 'ITSC':
        test_ind_file = 'data_test_ind_ITSC.csv'
    else:
        test_ind_file = 'data_test_ind_Ottawa.csv'

    if os.path.isfile(test_ind_file):
        indices = np.loadtxt(test_ind_file, delimiter=",").astype(int)
    else:
        indices = np.random.permutation(indices)
        np.savetxt(test_ind_file, indices, delimiter=",")

    data_train = data[indices[:div1]]
    data_val = data[indices[div1:div2]]
    data_test = data[indices[div2:]]

    labels_train = labels[indices[:div1]]
    labels_val = labels[indices[div1:div2]]
    labels_test = labels[indices[div2:]]

    return (data_train, data_val, data_test), (labels_train, labels_val, labels_test)

def dataloader_preparation(split_ratio=0.7, batch_size=50):
    global num_classes

    if case_study == 'CWRU':
        datasets = np.loadtxt('Vib_data2.csv', delimiter=",")
    elif case_study == 'ITSC':
        datasets = np.loadtxt('Iq_ITSC_PMSM.csv', delimiter=",")
    else:
        datasets = np.loadtxt('dataset_ottawa.csv', delimiter=",")

    num_features = datasets.shape[1] - 1
    num_classes = len(np.unique(datasets[:, -1]))

    data = datasets[:, np.newaxis, :num_features]
    labels = datasets[:, -1]

    data_split, labels_split = train_val_test_split(data, labels, split_ratio)

    dataloader_train = DataLoader(
        TensorDataset(torch.tensor(data_split[0], dtype=torch.float32), torch.tensor(labels_split[0], dtype=torch.long)),
        batch_size=batch_size,
        shuffle=True,
    )

    dataloader_val = DataLoader(
        TensorDataset(torch.tensor(data_split[1], dtype=torch.float32), torch.tensor(labels_split[1], dtype=torch.long)),
        batch_size=batch_size,
    )

    dataloader_test = DataLoader(
        TensorDataset(torch.tensor(data_split[2], dtype=torch.float32), torch.tensor(labels_split[2], dtype=torch.long)),
        batch_size=batch_size,
    )

    return dataloader_train, dataloader_val, dataloader_test

def train(dataloader, device, model, criterion, optimizer):
    model.train()
    total_loss, total_corrects, n_train = 0.0, 0, 0

    for batch in dataloader:
        inputs, labels = (t.to(device) for t in batch)
        labels = labels.long()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1, keepdim=True)
        total_corrects += preds.eq(labels.view_as(preds)).sum().item()
        total_loss += loss.item() * labels.size(0)
        n_train += labels.size(0)

    epoch_loss = total_loss / n_train
    epoch_acc = 100.0 * total_corrects / n_train
    print(f'Train loss: {epoch_loss:.4f} || Train accuracy: {epoch_acc:.2f}%')
    return epoch_loss, epoch_acc

def test(dataloader, device, model, criterion):
    model.eval()
    total_loss, total_corrects, n_test = 0.0, 0, 0

    with torch.no_grad():
        for batch in dataloader:
            inputs, labels = (t.to(device) for t in batch)
            labels = labels.long()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            preds = outputs.argmax(dim=1, keepdim=True)
            total_corrects += preds.eq(labels.view_as(preds)).sum().item()
            total_loss += loss.item() * labels.size(0)
            n_test += labels.size(0)

    epoch_loss = total_loss / n_test
    epoch_acc = 100.0 * total_corrects / n_test
    print(f'Test loss: {epoch_loss:.4f} || Test accuracy: {epoch_acc:.2f}%')
    return epoch_loss, epoch_acc

if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    log_file_name = f'results_logged_{case_study}_FSCNN.csv'
    n_epoch = 500
    batch_size = 32

    dataloader_train, dataloader_val, dataloader_test = dataloader_preparation(batch_size=batch_size)
    model = Net(n_class=num_classes, case_study=case_study).to(device).double()

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    best_acc = 0.0
    results = np.zeros((n_epoch, 6))

    for epoch in range(n_epoch):
        print(f'------------------ Epoch {epoch} ------------------')

        t0 = time.time()
        loss_train, acc_train = train(dataloader_train, device, model, criterion, optimizer)
        train_time = time.time() - t0

        loss_val, acc_val = test(dataloader_val, device, model, criterion)
        test_time = time.time() - t0 - train_time

        print(f'Training time: {train_time:.2f}s | Test time: {test_time:.2f}s')

        results[epoch, :] = [loss_train, acc_train, loss_val, acc_val, train_time, test_time]
        pd.DataFrame(results, columns=['loss_train', 'acc_train', 'loss_val', 'acc_val', 'train_time', 'test_time']).to_csv(log_file_name, index=False)

        if acc_val >= best_acc:
            best_acc = acc_val
            loss_test, acc_test = test(dataloader_test, device, model, criterion)

            if acc_train == 100 and acc_val == 100:
                model_path = f'./FSCNN_epoch_{case_study}_{epoch}.pt'
                torch.save(model.state_dict(), model_path)
                print(f'Model saved: {model_path}')